## 1. Introduction
<p><a href="https://commons.wikimedia.org/wiki/File:Tux.svg">
<img style="float: right;margin:5px 20px 5px 1px" width="150px" src="https://assets.datacamp.com/production/project_111/img/tux.png" alt="Tux: la mascota de Linux">
</a></p>
<p>Los repositorios de control de versiones como CVS, Subversion o Git pueden ser una verdadera mina de oro para los desarrolladores de software. Contienen cada cambio realizado al código fuente, incluyendo la fecha (el "cuándo"), el desarrollador responsable (el "quién"), así como un pequeño mensaje que describe la intención (el "qué") de un cambio.</p>
<p>En este cuaderno, analizaremos la evolución de un proyecto de código abierto muy famoso: el kernel de Linux. El kernel de Linux es el corazón de algunas distribuciones de Linux como Debian, Ubuntu o CentOS. Nuestro conjunto de datos contiene el historial del desarrollo del kernel durante casi 13 años (principios de 2005 - finales de 2017). Obtendremos información sobre el trabajo de los esfuerzos de desarrollo mediante:</p>
<ul>
<li>Identificar a los 10 principales contribuyentes </li>
<li>Visualizar los commits a lo largo de los años.</li>
</ul>

In [30]:
# Printing the content of git_log_excerpt.csv
# ... YOUR CODE FOR TASK 1 ...
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
df = pd.read_csv("datasets/git_log_excerpt.csv",sep="#",header=None,names=["timestamp","author"])
#Diez Principales contribuyentes
display(df.head(10))
#Imprimir que complete la tarea
print("Tarea Completada")

,timestamp,author
0,1502382966,Linus Torvalds
1,1501368308,Max Gurtovoy
2,1501625560,James Smart
3,1501625559,James Smart
4,1500568442,Martin Wilck
5,1502273719,Xin Long
6,1502278684,Nikolay Borisov
7,1502238384,Girish Moodalbail
8,1502228709,Florian Fainelli
9,1502223836,Jon Paul Maloy


Tarea Completada


## 2. Reading in the dataset
<p>El conjunto de datos fue creado utilizando el comando <code>git log --encoding=latin-1 --pretty="%at#%aN"</code> a finales de 2017. La salida de texto codificada en <code>latin-1</code> se guardó en un archivo CSV sin encabezado. En este archivo, cada fila es una entrada de commit con la siguiente información:</p>
<ul>
<li><code>timestamp</code>: la hora del commit como una marca de tiempo UNIX en segundos desde 1970-01-01 00:00:00 (Marcador de posición de Git log "<code>%at</code>")</li>
<li><code>author</code>: el nombre del autor que realizó el commit (Marcador de posición de Git log "<code>%aN</code>")</li>
</ul>
<p>Las columnas están separadas por el signo de número <code>#</code>. El conjunto de datos completo se encuentra en el directorio <code>datasets/</code>. Es un archivo csv comprimido en <code>gz</code> llamado <code>git_log.gz</code>.</p>

In [31]:
# Loading in the pandas module as 'pd'
# ... YOUR CODE FOR TASK 2 ...

# Reading in the log file
git_log = pd.read_csv("datasets/git_log_data.csv")

# Printing out the first 5 rows
git_log.head()

,timestamp,author
0,1502826583,Linus Torvalds
1,1501749089,Adrian Hunter
2,1501749088,Adrian Hunter
3,1501882480,Kees Cook
4,1497271395,Rob Clark


## 3. Obteniendo una visión general
<p>El conjunto de datos contiene información sobre cada contribución de código (un "commit") al kernel de Linux en los últimos 13 años. Primero echaremos un vistazo al número de autores y sus commits en el repositorio.</p>

In [32]:
# calculating number of commits
number_of_commits = len(git_log)

# calculating number of authors
number_of_authors = git_log["author"].nunique()

# printing out the results
print("%s authors committed %s code changes." % (number_of_authors, number_of_commits))

17385 authors committed 699071 code changes.


## 4. Encontrando los 10 principales contribuyentes
<p>Existen algunas personas muy importantes que han realizado cambios frecuentes en el kernel de Linux. Para ver si hay algún cuello de botella, vamos a analizar a los 10 principales autores con más commits.</p>

In [33]:
# Identifying the top 10 authors
top_10_authors = git_log["author"].value_counts().head(10)

# Listing contents of 'top_10_authors'
top_10_authors

Linus Torvalds           23361
David S. Miller           9106
Mark Brown                6802
Takashi Iwai              6209
Al Viro                   6006
H Hartley Sweeten         5938
Ingo Molnar               5344
Mauro Carvalho Chehab     5204
Arnd Bergmann             4890
Greg Kroah-Hartman        4580
Name: author, dtype: int64

## 5. Manipulando los datos
<p>Para nuestro análisis, queremos visualizar las contribuciones a lo largo del tiempo. Para esto, utilizaremos la información en la columna <code>timestamp</code> para crear una columna basada en series de tiempo.</p>

In [34]:
# converting the timestamp column
# ... YOUR CODE FOR TASK 5 ...
git_log["timestamp"] = pd.to_datetime(git_log["timestamp"],unit="s")
# summarizing the converted timestamp column
# ... YOUR CODE FOR TASK 5 ...
git_log.describe()

,timestamp,author
count,699071,699070
unique,668448,17385
top,2008-09-04 05:30:19,Linus Torvalds
freq,99,23361
first,1970-01-01 00:00:01,NaN
last,2037-04-25 08:08:26,NaN


## 6. Tratando los timestamps incorrectos
<p>Como podemos ver en los resultados anteriores, algunos contribuyentes tenían incorrectamente configurada la hora de su sistema operativo cuando realizaron los commits en el repositorio. Vamos a limpiar la columna <code>timestamp</code> eliminando las filas con timestamps incorrectos.</p>

In [35]:
#Obtenemos Primer Día del 2005
principio_2005 = pd.to_datetime("2005")

#Filtramos que nuestros commits sean de ese año hacia delante.
git_log_filtrado = git_log[git_log["timestamp"] >= principio_2005]

# determining the first real commit timestamp
first_commit_timestamp = git_log_filtrado["timestamp"].min()

# determining the last sensible commit timestamp
last_commit_timestamp = pd.to_datetime("2018")

# filtering out wrong timestamps
corrected_log = git_log[(git_log["timestamp"] >= first_commit_timestamp) & (git_log_filtrado["timestamp"]<= last_commit_timestamp)]

# summarizing the corrected timestamp column
# ... YOUR CODE FOR TASK 6 ...

## 7. Agrupando los commits por año
<p>Para determinar cómo ha aumentado la actividad de desarrollo a lo largo del tiempo, vamos a agrupar los commits por año y contarlos.</p>

In [42]:
corrected_log["author"] = corrected_log["author"].fillna("Unknown")
# Counting the no. commits per year
corrected_log["Year"] = corrected_log["timestamp"].dt.year
# ... YOUR CODE FOR TASK 7 ...
commit_per_year = corrected_log.groupby(["Year","empresa"]).count().reset_index()

## 8. Visualizando la historia de Linux
<p>Finalmente, crearemos un gráfico a partir de estos recuentos para poder visualizar mejor cómo ha aumentado el esfuerzo de desarrollo en Linux en los últimos años.</p>

## 9. Conclusión
<p>Gracias a la sólida base y al cuidado de Linux Torvalds, muchos otros desarrolladores ahora también pueden contribuir al kernel de Linux. ¡No se observa una disminución de la actividad de desarrollo a la vista!</p>

In [38]:
# calculating or setting the year with the most commits to Linux
year_with_most_commits = 2016 